In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.cluster import KMeans

In [2]:
nba_data = pd.read_csv('Data/nba_data.csv')
players = nba_data.player.to_list()


In [3]:
nba_data = nba_data.set_index('rank')
nba_data.drop(['Unnamed: 0', 'player', 'pos', 'team_id', 'age', 'mp', 'fg', 'fga','fg3',
 'fg3a','fg2',
 'fg2a','ft',
 'fta','orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',], axis=1, inplace=True)
list(nba_data.columns.values)


['g',
 'gs',
 'fg_pct',
 'fg3_pct',
 'fg2_pct',
 'efg_pct',
 'ft_pct',
 'mp_per_g',
 'fg_per_g',
 'fga_per_g',
 'fg3_per_g',
 'fg3a_per_g',
 'fg2_per_g',
 'fg2a_per_g',
 'ft_per_g',
 'fta_per_g',
 'orb_per_g',
 'drb_per_g',
 'trb_per_g',
 'ast_per_g',
 'stl_per_g',
 'blk_per_g',
 'tov_per_g',
 'pf_per_g',
 'pts_per_g',
 'fg_per_mp',
 'fga_per_mp',
 'fg3_per_mp',
 'fg3a_per_mp',
 'fg2_per_mp',
 'fg2a_per_mp',
 'ft_per_mp',
 'fta_per_mp',
 'orb_per_mp',
 'drb_per_mp',
 'trb_per_mp',
 'ast_per_mp',
 'stl_per_mp',
 'blk_per_mp',
 'tov_per_mp',
 'pf_per_mp',
 'pts_per_mp',
 'fg_per_poss',
 'fga_per_poss',
 'fg3_per_poss',
 'fg3a_per_poss',
 'fg2_per_poss',
 'fg2a_per_poss',
 'ft_per_poss',
 'fta_per_poss',
 'orb_per_poss',
 'drb_per_poss',
 'trb_per_poss',
 'ast_per_poss',
 'stl_per_poss',
 'blk_per_poss',
 'tov_per_poss',
 'pf_per_poss',
 'pts_per_poss',
 'off_rtg',
 'def_rtg',
 'per',
 'ts_pct',
 'fg3a_per_fga_pct',
 'fta_per_fga_pct',
 'orb_pct',
 'drb_pct',
 'trb_pct',
 'ast_pct',
 'stl

In [4]:
nba_data
offprofile_df = nba_data[['fg_pct', 'fg3_pct', 'fg2_pct', 'efg_pct', 'ft_pct','ts_pct', 'fg3a_per_fga_pct', 
                           'fta_per_fga_pct', 'fg_per_poss', 'fga_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 
                          'fg2_per_poss', 'fg2a_per_poss', 'ft_per_poss', 'fta_per_poss', 'ast_per_poss', 'pts_per_poss', 
                          'usg_pct', 'ast_pct', 'tov_pct', 'orb_pct']]

In [5]:
nba_data_minus_games = nba_data.drop(['g',
 'gs'], axis=1)

nba_data_minus_games

,fg_pct,fg3_pct,fg2_pct,efg_pct,ft_pct,mp_per_g,fg_per_g,fga_per_g,fg3_per_g,fg3a_per_g,...,tov_pct,usg_pct,ows,dws,ws,ws_per_48,obpm,dbpm,bpm,vorp
rank,,,,,,,,,,,,,,,,,,,,,
1,0.357,0.323,0.500,0.487,0.923,19.0,1.8,5.1,1.3,4.1,...,7.9,12.2,0.1,0.6,0.6,0.053,-2.4,-0.9,-3.4,-0.2
3,0.345,0.338,0.361,0.459,0.778,12.6,1.1,3.2,0.7,2.2,...,19.7,13.5,-0.1,0.2,0.1,0.011,-3.1,-1.3,-4.4,-0.3
4,0.595,0.000,0.596,0.595,0.500,33.4,6.0,10.1,0.0,0.0,...,12.6,16.4,5.1,4.0,9.1,0.163,0.6,2.1,2.7,3.2
5,0.576,0.200,0.588,0.579,0.735,23.3,3.4,5.9,0.0,0.2,...,17.1,15.8,3.4,3.4,6.8,0.171,-0.6,3.6,3.0,2.4
8,0.519,0.238,0.528,0.522,0.847,33.2,8.4,16.3,0.1,0.5,...,8.8,26.9,6.4,2.9,9.3,0.167,1.2,0.5,1.6,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,0.422,0.341,0.515,0.513,0.748,27.2,2.8,6.6,1.2,3.5,...,11.5,13.2,0.3,2.0,2.2,0.052,-1.7,0.6,-1.2,0.4
488,0.559,0.292,0.586,0.572,0.795,22.3,6.1,11.0,0.3,1.0,...,12.7,28.5,2.2,1.9,4.2,0.183,0.1,1.2,1.2,0.9
521,0.521,0.346,0.571,0.560,0.732,12.0,2.9,5.6,0.4,1.2,...,10.7,26.1,0.4,0.3,0.8,0.151,-1.5,-1.8,-3.3,-0.1


In [6]:
nba_data_minus_advanced = nba_data.drop(['per',
 'ts_pct',
 'fg3a_per_fga_pct',
 'fta_per_fga_pct',
 'orb_pct',
 'drb_pct',
 'trb_pct',
 'ast_pct',
 'stl_pct',
 'blk_pct',
 'tov_pct',
 'usg_pct',
 'ows',
 'dws',
 'ws',
 'ws_per_48',
 'obpm',
 'dbpm',
 'bpm',
 'vorp',
 'off_rtg',
 'def_rtg'], axis=1)

In [7]:
nba_advanced_data = nba_data.drop(['g',
 'gs',
    'fg_pct',
 'fg3_pct',
 'fg2_pct',
 'efg_pct',
 'ft_pct',
 'mp_per_g',
 'fg_per_g',
 'fga_per_g',
 'fg3_per_g',
 'fg3a_per_g',
 'fg2_per_g',
 'fg2a_per_g',
 'ft_per_g',
 'fta_per_g',
 'orb_per_g',
 'drb_per_g',
 'trb_per_g',
 'ast_per_g',
 'stl_per_g',
 'blk_per_g',
 'tov_per_g',
 'pf_per_g',
 'pts_per_g',
 'fg_per_mp',
 'fga_per_mp',
 'fg3_per_mp',
 'fg3a_per_mp',
 'fg2_per_mp',
 'fg2a_per_mp',
 'ft_per_mp',
 'fta_per_mp',
 'orb_per_mp',
 'drb_per_mp',
 'trb_per_mp',
 'ast_per_mp',
 'stl_per_mp',
 'blk_per_mp',
 'tov_per_mp',
 'pf_per_mp',
 'pts_per_mp',
 'fg_per_poss',
 'fga_per_poss',
 'fg3_per_poss',
 'fg3a_per_poss',
 'fg2_per_poss',
 'fg2a_per_poss',
 'ft_per_poss',
 'fta_per_poss',
 'orb_per_poss',
 'drb_per_poss',
 'trb_per_poss',
 'ast_per_poss',
 'stl_per_poss',
 'blk_per_poss',
 'tov_per_poss',
 'pf_per_poss',
 'pts_per_poss',], axis=1)

In [8]:
pd.options.display.max_rows = 100

def run_cluster(data, num_clusters=5):
    # create kmeans model
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(data)

    # find centers and labels
    centers = kmeans.cluster_centers_
    labels = kmeans.labels_

    # DataFrame of centroids
    centroids_df = pd.DataFrame(centers, columns=data.columns).T

    # print items in clusters
    clusters = {}
    for i in range(len(labels)):
        clusters[players[i]] = labels[i]

    for i in range(len(centers)):
        out = [k for k,v in clusters.items() if v==i]

        print('-'*30)
        print(f'cluster index: {i}')
        for item in out:
            print(item)
        print('\n')

    centroids_df.to_csv('test3.csv')
    return centroids_df

In [9]:
run_cluster(offprofile_df, 7)

------------------------------
cluster index: 0
Justin Anderson
Carmelo Anthony
Dwayne Bacon
Will Barton
Malik Beasley
Marco Belinelli
Nemanja Bjelica
Dillon Brooks
Kentavious Caldwell-Pope
DeMarre Carroll
Vince Carter
Omri Casspi
Ian Clark
Quinn Cook
Jae Crowder
Seth Curry
Troy Daniels
Dewayne Dedmon
Luol Deng
Damyean Dotson
Yogi Ferrell
Bryn Forbes
Channing Frye
Langston Galloway
Eric Gordon
Jerami Grant
Gerald Green
Jeff Green
Devin Harris
Gary Harris
Joe Harris
Mario Hezonja
Frank Jackson
Jalen Jones
Luke Kennard
Kevin Knox
Furkan Korkmaz
Luke Kornet
Rodions Kurucs
Jake Layman
Damion Lee
Brook Lopez
Doug McDermott
Patty Mills
E'Twaun Moore
Marcus Morris
Georges Niang
Dirk Nowitzki
Kelly Olynyk
Cedi Osman
Chandler Parsons
Theo Pinson
Norman Powell
Jerome Robinson
Terry Rozier
J.R. Smith
Moritz Wagner
Dion Waiters
Taurean Waller-Prince
Trevor Ariza
Harrison Barnes
Reggie Bullock
Alec Burks
Marquese Chriss
Tyler Dorsey
Henry Ellenson
Wayne Ellington
JaMychal Green
Rodney Hood
John Jen

,0,1,2,3,4,5,6
fg_pct,0.424988,0.421087,0.444660,0.465838,0.537333,0.428776,0.552512
fg3_pct,0.360277,0.337870,0.353396,0.338000,0.166902,0.323463,0.222585
fg2_pct,0.489699,0.508543,0.492415,0.517432,0.564588,0.476269,0.575902
efg_pct,0.516952,0.511326,0.508792,0.520351,0.555039,0.488552,0.566463
ft_pct,0.792398,0.732772,0.798415,0.797324,0.659373,0.757045,0.700366
ts_pct,0.546747,0.536011,0.547623,0.568162,0.580353,0.520866,0.596634
fg3a_per_fga_pct,0.509108,0.526043,0.359774,0.312270,0.118529,0.354836,0.089122
fta_per_fga_pct,0.194337,0.188283,0.250208,0.326865,0.355804,0.224239,0.362585
fg_per_poss,7.286747,5.335870,9.566038,11.805405,6.627451,6.686567,9.931707
fga_per_poss,17.207229,12.672826,21.533962,25.370270,12.498039,15.562687,18.075610


In [10]:
run_cluster(nba_data_minus_games, 7)

------------------------------
cluster index: 0
J.J. Barea
Bradley Beal
Eric Bledsoe
Devin Booker
Mike Conley
Stephen Curry
DeMar DeRozan
Luka Doncic
Kevin Durant
De'Aaron Fox
Paul George
Blake Griffin
James Harden
Jrue Holiday
Kyrie Irving
LeBron James
Zach LaVine
Kawhi Leonard
Damian Lillard
Donovan Mitchell
Victor Oladipo
Chris Paul
Derrick Rose
D'Angelo Russell
Ben Simmons
Kemba Walker
John Wall
Russell Westbrook
Lou Williams
Trae Young


------------------------------
cluster index: 1
Al-Farouq Aminu
Kyle Anderson
OG Anunoby
Ryan Arcidiacono
Dwayne Bacon
Nicolas Batum
Malik Beasley
Marco Belinelli
Davis Bertans
Patrick Beverley
Nemanja Bjelica
Jaron Blossomgame
Jonah Bolden
Mikal Bridges
Miles Bridges
Ryan Broekhoff
Sterling Brown
Bruno Caboclo
Kentavious Caldwell-Pope
DeMarre Carroll
Vince Carter
Gary Clark
Pat Connaughton
Allen Crabbe
Torrey Craig
Jae Crowder
Dante Cunningham
Seth Curry
Luol Deng
Damyean Dotson
Jared Dudley
Terrance Ferguson
Dorian Finney-Smith
Bryn Forbes
Langs

,0,1,2,3,4,5,6
fg_pct,0.458700,0.440273,0.390188,0.541478,0.553412,0.430397,0.446045
fg3_pct,0.334700,0.358955,0.300174,0.251174,0.180544,0.340690,0.355773
fg2_pct,0.511100,0.517545,0.458870,0.569870,0.579265,0.481224,0.494803
efg_pct,0.518867,0.535427,0.459594,0.562130,0.570706,0.495034,0.512258
ft_pct,0.798533,0.767000,0.724812,0.738174,0.686544,0.762966,0.785045
mp_per_g,33.006667,21.656364,14.884058,29.247826,17.336765,21.765517,28.409091
fg_per_g,7.940000,2.811818,1.810145,7.091304,2.808824,3.124138,5.471212
fga_per_g,17.336667,6.408182,4.624638,13.239130,5.097059,7.255172,12.277273
fg3_per_g,2.150000,1.234545,0.643478,0.595652,0.188235,0.934483,1.657576
fg3a_per_g,6.020000,3.360909,2.086957,1.869565,0.576471,2.687931,4.553030


In [11]:
run_cluster(nba_data_minus_advanced, 7)

------------------------------
cluster index: 0
Ryan Arcidiacono
Kent Bazemore
Patrick Beverley
Mikal Bridges
Bruce Brown
Jalen Brunson
Thomas Bryant
Torrey Craig
Dewayne Dedmon
Damyean Dotson
Rudy Gay
Taj Gibson
Eric Gordon
Jeff Green
Maurice Harkless
Gary Harris
Kevin Huerter
Serge Ibaka
Jaren Jackson
Kevin Knox
Rodions Kurucs
Jeremy Lamb
Jake Layman
Alex Len
Robin Lopez
Rodney McGruder
Marcus Morris
Larry Nance
Josh Okogie
Kelly Olynyk
Chris Paul
Tomas Satoransky
Marcus Smart
Noah Vonleh
Derrick White
Hassan Whiteside
Justise Winslow
Avery Bradley
Reggie Bullock
Rodney Hood
Dario Saric
Iman Shumpert
Garrett Temple


------------------------------
cluster index: 1
Justin Anderson
Dwayne Bacon
Mo Bamba
Aron Baynes
Jordan Bell
Khem Birch
Antonio Blakeney
Jonah Bolden
Ryan Broekhoff
Sterling Brown
Troy Brown
Jose Calderon
Gary Clark
Ian Clark
Pat Connaughton
Dante Cunningham
Troy Daniels
Cheick Diallo
Hamidou Diallo
Jared Dudley
Dante Exum
Cristiano Felicio
Harry Giles
Devonte' Graham
J

,0,1,2,3,4,5,6
g,71.023256,52.511364,72.704545,48.224490,73.127907,27.640625,76.52000
gs,48.651163,5.625000,71.954545,33.469388,10.604651,3.718750,74.56000
fg_pct,0.457023,0.451205,0.475795,0.455286,0.458314,0.430281,0.48094
fg3_pct,0.338721,0.278977,0.330477,0.296592,0.321942,0.302938,0.32538
fg2_pct,0.519512,0.511591,0.522432,0.501204,0.516930,0.486406,0.52998
efg_pct,0.525581,0.510682,0.530750,0.511469,0.528477,0.494297,0.54788
ft_pct,0.768581,0.722670,0.807000,0.735816,0.759651,0.719797,0.76924
mp_per_g,25.565116,14.504545,33.311364,25.734694,21.347674,14.895313,29.93600
fg_per_g,3.855814,1.964773,8.109091,4.351020,3.354651,2.010938,4.85800
fga_per_g,8.481395,4.430682,17.127273,9.655102,7.410465,4.679688,10.16400


In [12]:
run_cluster(nba_advanced_data, 7)

------------------------------
cluster index: 0
Alex Abrines
Jaylen Adams
Grayson Allen
Justin Anderson
Carmelo Anthony
OG Anunoby
Will Barton
Keita Bates-Diop
Kent Bazemore
Michael Beasley
DeAndre' Bembry
Dragan Bender
Antonio Blakeney
Isaiah Briscoe
Dillon Brooks
MarShon Brooks
Bruce Brown
Jaylen Brown
Lorenzo Brown
Troy Brown
Deonte Burton
Jevon Carter
Ian Clark
Allen Crabbe
Troy Daniels
Hamidou Diallo
Donte DiVincenzo
Jacob Evans
Tyreke Evans
Channing Frye
Markelle Fultz
Harry Giles
Treveon Graham
Daniel Hamilton
Shaquille Harrison
Josh Hart
Mario Hezonja
Solomon Hill
Rondae Hollis-Jefferson
Chandler Hutchison
Brandon Ingram
Frank Jackson
Josh Jackson
James Johnson
Kevin Knox
Rodions Kurucs
Trey Lyles
Rodney McGruder
De'Anthony Melton
Malik Monk
Abdel Nader
Dirk Nowitzki
Frank Ntilikina
Semi Ojeleye
Elie Okobo
Josh Okogie
Chandler Parsons
Theo Pinson
Glenn Robinson
Jerome Robinson
Brandon Sampson
Collin Sexton
J.R. Smith
Lance Stephenson
Edmond Sumner
Lance Thomas
Sindarius Thornwe

,0,1,2,3,4,5,6
off_rtg,98.096774,111.528571,113.913793,114.64000,112.732558,127.857143,107.406250
def_rtg,112.236559,113.300000,107.827586,109.04000,110.000000,107.178571,111.562500
per,9.255914,12.595714,18.924138,22.40000,12.503488,19.182143,13.887500
ts_pct,0.494849,0.561414,0.581345,0.57848,0.562977,0.644179,0.527250
fg3a_per_fga_pct,0.428892,0.498414,0.151448,0.35008,0.435267,0.092750,0.350078
fta_per_fga_pct,0.198699,0.190300,0.340672,0.33028,0.233477,0.417786,0.238281
orb_pct,2.938710,2.181429,9.724138,3.44400,5.418605,10.778571,2.250000
drb_pct,12.813978,10.572857,25.765517,16.47600,16.072093,20.700000,10.893750
trb_pct,7.836559,6.360000,17.734483,10.01200,10.754651,15.750000,6.579688
ast_pct,10.743011,11.522857,11.989655,31.67600,8.196512,9.157143,25.920313
